In [29]:
#import libraries
from bs4 import BeautifulSoup
from requests import get
import pandas as pd
import re

In [30]:
#download episode data

url = 'https://www.imdb.com/title/tt0436992/episodes?season=1'
response = get(url)
print (response.text[:250])


 










<!DOCTYPE html>
<html
    xmlns:og="http://ogp.me/ns#"
    xmlns:fb="http://www.facebook.com/2008/fbml">
    <head>
         

        <meta charset="utf-8">
        <meta http-equiv="X-UA-Compatible" content="IE=edge">




        <scri


In [31]:
html_soup = BeautifulSoup(response.text, 'html.parser')
type(html_soup)

bs4.BeautifulSoup

In [32]:
#extract episode data
episode_containers = html_soup.find_all('div', class_='info')

In [37]:
episode_containers[0].a['title']

episode_containers[0].meta['content']

episode_containers[0].find('div', class_='airdate').text.strip()

episode_containers[0].find('span', class_='ipl-rating-star__rating').text

episode_containers[0].find('span', class_='ipl-rating-star__total-votes').text

'(8,533)'

In [38]:
#episode description
episode_containers[0].find('div', class_='item_description').text.strip()

'When ordinary shop-worker Rose Tyler meets a mysterious stranger called the Doctor she is drawn into his strange and dangerous world; her life will never be the same again.'

In [39]:
#need to create a loop for all seasons
#initializing series the loop will populate
doctor_who_episodes = []

#for every season in series
for sn in range (1,13):
    response = get('https://www.imdb.com/title/tt0436992/episodes?season=' + str(sn))
    #parse content
    page_html = BeautifulSoup(response.text, 'html.parser')
    #find all div containers
    episode_containers = page_html.find_all('div', class_ = 'info')
    #for each episode in each season
    for episodes in episode_containers:
        season = sn
        episode_number = episodes.meta['content']
        title = episodes.a['title']
        airdate = episodes.find('div', class_ = 'airdate').text.strip()
        rating = episodes.find ('span', class_ = 'ipl-rating-star__rating').text
        total_votes = episodes.find ('span', class_ = 'ipl-rating-star__total-votes').text
        desc = episodes.find ('div', class_= 'item_description').text.strip()
        #compiling episode info
        episode_data = [season, episode_number, title, airdate, rating, total_votes, desc]
        #append info to complete dataset
        doctor_who_episodes.append(episode_data)

In [40]:
#need to create a loop for all seasons
#initializing series the loop will populate
dw_episodes_unknown = []


#remove first for loop to scrape page of episodes listed as Unknown
response = get('https://www.imdb.com/title/tt0436992/episodes?season=-1')
page_html = BeautifulSoup(response.text, 'html.parser')
episode_containers_2 = page_html.find_all('div', class_ = 'info')
for episodes in episode_containers_2:
    season = sn
    episode_number = episodes.meta['content']
    title = episodes.a['title']
    airdate = episodes.find('div', class_ = 'airdate').text.strip()
    rating = episodes.find ('span', class_ = 'ipl-rating-star__rating').text
    total_votes = episodes.find ('span', class_ = 'ipl-rating-star__total-votes').text
    desc = episodes.find ('div', class_= 'item_description').text.strip()
    episode_data = [season, episode_number, title, airdate, rating, total_votes, desc]
    dw_episodes_unknown.append(episode_data)

In [41]:
#make the dataframes for both
doctor_who_episodes = pd.DataFrame (doctor_who_episodes , columns = ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc'])

doctor_who_episodes.head()


,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Rose,17 Mar. 2006,7.5,"(8,533)",When ordinary shop-worker Rose Tyler meets a m...
1,1,2,The End of the World,17 Mar. 2006,7.5,"(7,471)",The Doctor takes Rose to the year 5 billion to...
2,1,3,The Unquiet Dead,24 Mar. 2006,7.5,"(6,986)",The Doctor has great expectations for his late...
3,1,4,Aliens of London,31 Mar. 2006,7.0,"(6,718)",The Doctor returns Rose to her own time - well...
4,1,5,World War Three,7 Apr. 2006,7.0,"(6,514)",The Slitheen have infiltrated Parliament and h...


In [42]:
dw_episodes_unknown = pd.DataFrame (dw_episodes_unknown , columns = ['season', 'episode_number', 'title', 'airdate', 'rating', 'total_votes', 'desc'])

dw_episodes_unknown.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,12,-1,Time Crash,16 Nov. 2007,8.4,"(3,112)",The Tenth Doctor meets one of his previous inc...
1,12,-1,The Next Doctor,27 Jun. 2009,7.5,"(4,602)",The Doctor arrives in London on Christmas Eve ...
2,12,-1,Planet of the Dead,26 Jul. 2009,7.5,"(4,565)",A meeting in a London bus with jewel thief Lad...
3,12,-1,The Waters of Mars,19 Dec. 2009,8.7,"(5,831)",In a Mars base the inhabitants are being infec...
4,12,-1,A Ghost Story for Christmas,24 Dec. 2009,7.2,(586),This mini-story tells of the Weeping Angels an...


In [43]:
#combine df into one dataset
dw_episodes = [doctor_who_episodes, dw_episodes_unknown]

dw_episodes_new = pd.concat(dw_episodes)
display(dw_episodes_new)

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Rose,17 Mar. 2006,7.5,"(8,533)",When ordinary shop-worker Rose Tyler meets a m...
1,1,2,The End of the World,17 Mar. 2006,7.5,"(7,471)",The Doctor takes Rose to the year 5 billion to...
2,1,3,The Unquiet Dead,24 Mar. 2006,7.5,"(6,986)",The Doctor has great expectations for his late...
3,1,4,Aliens of London,31 Mar. 2006,7.0,"(6,718)",The Doctor returns Rose to her own time - well...
4,1,5,World War Three,7 Apr. 2006,7.0,"(6,514)",The Slitheen have infiltrated Parliament and h...
...,...,...,...,...,...,...,...
24,12,-1,The Night of the Doctor,14 Nov. 2013,9.1,"(3,649)","With the aid of the Sisterhood of Karn, the Ei..."
25,12,-1,The Day of the Doctor,23 Nov. 2013,9.3,"(18,006)","In 2013, something terrible is awakening in Lo..."
26,12,-1,Prologue,11 Sep. 2015,7.8,(570),Doctor Who Season 9 starts here with this prol...
27,12,-1,The Doctor's Meditation,15 Sep. 2015,7.7,(646),The Doctor contemplates a journey to see an ol...


In [44]:
#cleaning the data
#remove  ‘,’ , ‘(', and ‘)’ from total_votes so we can convert data type
def remove_str(votes):
    for r in ((',',''), ('(',''),(')','')):
        votes = votes.replace(*r)
        
    return votes

In [45]:
#convert total_votes to int
dw_episodes_new['total_votes'] = dw_episodes_new.total_votes.apply(remove_str).astype(int)

dw_episodes_new.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Rose,17 Mar. 2006,7.5,8533,When ordinary shop-worker Rose Tyler meets a m...
1,1,2,The End of the World,17 Mar. 2006,7.5,7471,The Doctor takes Rose to the year 5 billion to...
2,1,3,The Unquiet Dead,24 Mar. 2006,7.5,6986,The Doctor has great expectations for his late...
3,1,4,Aliens of London,31 Mar. 2006,7.0,6718,The Doctor returns Rose to her own time - well...
4,1,5,World War Three,7 Apr. 2006,7.0,6514,The Slitheen have infiltrated Parliament and h...


In [50]:
#change rating from string to numeric
dw_episodes_new['rating'] = dw_episodes_new.rating.astype(float)

In [51]:
#convert airdate to datetime format
dw_episodes_new['airdate'] = pd.to_datetime (dw_episodes_new.airdate)

dw_episodes_new.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 189 entries, 0 to 28
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   season          189 non-null    int64         
 1   episode_number  189 non-null    object        
 2   title           189 non-null    object        
 3   airdate         189 non-null    datetime64[ns]
 4   rating          189 non-null    float64       
 5   total_votes     189 non-null    int64         
 6   desc            189 non-null    object        
dtypes: datetime64[ns](1), float64(1), int64(2), object(3)
memory usage: 11.8+ KB


In [52]:
dw_episodes_new.head()

,season,episode_number,title,airdate,rating,total_votes,desc
0,1,1,Rose,2006-03-17,7.5,8533,When ordinary shop-worker Rose Tyler meets a m...
1,1,2,The End of the World,2006-03-17,7.5,7471,The Doctor takes Rose to the year 5 billion to...
2,1,3,The Unquiet Dead,2006-03-24,7.5,6986,The Doctor has great expectations for his late...
3,1,4,Aliens of London,2006-03-31,7.0,6718,The Doctor returns Rose to her own time - well...
4,1,5,World War Three,2006-04-07,7.0,6514,The Slitheen have infiltrated Parliament and h...


In [53]:
dw_episodes_new.to_csv('dw_episodes_imdb_ratings.csv',index=False)